# Downloading Sentinel-2 and Landsat Data

This notebook demonstrates how to download Sentinel-2 imagery using `sentinelsat` and Landsat imagery using `landsatexplorer` in Python. It includes querying data for a specified area of interest (AOI) and time range, downloading products, and visualizing RGB composites with `rasterio` and `matplotlib`.

## Prerequisites
- Install required libraries: `sentinelsat`, `landsatexplorer`, `rasterio`, `numpy`, `matplotlib`, `geopandas` (listed in `requirements.txt`).
- A Copernicus Open Access Hub account for Sentinel-2 (register at https://scihub.copernicus.eu/).
- An EarthExplorer account for Landsat (register at https://earthexplorer.usgs.gov/).
- A GeoJSON file defining the area of interest (e.g., `aoi.geojson`). Replace file paths with your own data.
- Ensure sufficient disk space (~1-2 GB per Sentinel-2 product, ~500 MB per Landsat scene).

## Learning Objectives
- Query and download Sentinel-2 and Landsat imagery for a specified AOI and time range.
- Load and visualize RGB bands from downloaded products.
- Save RGB composites as GeoTIFF files for further analysis.

In [ ]:
# Import required libraries
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from landsatexplorer.ee_api import EarthExplorer
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from datetime import date
import glob

## Step 1: Set Up APIs and Query Sentinel-2 Data

Connect to the Copernicus Open Access Hub and query Sentinel-2 imagery for the specified AOI and time range.

In [ ]:
# Initialize SentinelAPI with your credentials
sentinel_api = SentinelAPI('your_scihub_username', 'your_scihub_password', 'https://scihub.copernicus.eu/dhus')

# Define area of interest (AOI) and query parameters
aoi_path = 'aoi.geojson'  # Path to your GeoJSON file
footprint = geojson_to_wkt(read_geojson(aoi_path))
start_date = date(2025, 1, 1)
end_date = date(2025, 6, 30)

# Query Sentinel-2 Level-2A products
sentinel_products = sentinel_api.query(
    footprint,
    date=(start_date, end_date),
    platformname='Sentinel-2',
    producttype='S2MS2Ap',
    cloudcoverpercentage=(0, 30)  # Limit to images with <30% cloud cover
)

# Convert to DataFrame and sort by cloud cover
sentinel_products_df = sentinel_api.to_dataframe(sentinel_products)
sentinel_products_df = sentinel_products_df.sort_values('cloudcoverpercentage')

# Print query results
print(f'Found {len(sentinel_products_df)} Sentinel-2 products')
print(sentinel_products_df[['title', 'cloudcoverpercentage', 'ingestiondate']].head())

## Step 2: Download Sentinel-2 Product

Download the Sentinel-2 product with the lowest cloud cover.

In [ ]:
# Select the first Sentinel-2 product (lowest cloud cover)
sentinel_product_id = sentinel_products_df.index[0]
sentinel_product_title = sentinel_products_df.loc[sentinel_product_id, 'title']

# Define output directory
output_dir = 'remote_sensing_data/'
os.makedirs(output_dir, exist_ok=True)

# Download Sentinel-2 product
sentinel_api.download(sentinel_product_id, directory_path=output_dir)
sentinel_product_path = os.path.join(output_dir, f'{sentinel_product_title}.SAFE')

print(f'Downloaded Sentinel-2 product: {sentinel_product_path}')

## Step 3: Query and Download Landsat Data

Connect to EarthExplorer and download a Landsat 8/9 scene for the same AOI and time range.

In [ ]:
# Initialize EarthExplorer API
ee = EarthExplorer('your_earthexplorer_username', 'your_earthexplorer_password')

# Load AOI and convert to WKT
aoi_gdf = gpd.read_file(aoi_path)
aoi_wkt = aoi_gdf.geometry.to_wkt().iloc[0]

# Query Landsat 8/9 Collection 2 Level-2 products
landsat_products = ee.search(
    dataset='LANDSAT_8_C2_L2' if date(2025, 1, 1) > date(2022, 5, 1) else 'LANDSAT_9_C2_L2',
    start_date='2025-01-01',
    end_date='2025-06-30',
    wkt=aoi_wkt,
    max_cloud_cover=30
)

# Convert to DataFrame and sort by cloud cover
landsat_products_df = pd.DataFrame(landsat_products)
landsat_products_df = landsat_products_df.sort_values('cloud_cover')

# Print query results
print(f'Found {len(landsat_products_df)} Landsat products')
print(landsat_products_df[['display_id', 'cloud_cover', 'acquisition_date']].head())

In [ ]:
# Select and download the first Landsat product
landsat_product_id = landsat_products_df['display_id'].iloc[0]
landsat_product_path = ee.download(
    display_id=landsat_product_id,
    output_dir=output_dir,
    dataset='LANDSAT_8_C2_L2' if date(2025, 1, 1) > date(2022, 5, 1) else 'LANDSAT_9_C2_L2'
)

print(f'Downloaded Landsat product: {landsat_product_path}')

## Step 4: Visualize Sentinel-2 RGB Composite

Load and visualize the RGB bands (B4, B3, B2) from the downloaded Sentinel-2 product.

In [ ]:
# Define paths to Sentinel-2 RGB bands (10m resolution)
sentinel_granule_path = os.path.join(sentinel_product_path, 'GRANULE', '*', 'IMG_DATA', 'R10m')
red_path = glob.glob(os.path.join(sentinel_granule_path, '*_B04_10m.jp2'))[0]
green_path = glob.glob(os.path.join(sentinel_granule_path, '*_B03_10m.jp2'))[0]
blue_path = glob.glob(os.path.join(sentinel_granule_path, '*_B02_10m.jp2'))[0]

# Load Sentinel-2 bands
with rasterio.open(red_path) as src_red, rasterio.open(green_path) as src_green, rasterio.open(blue_path) as src_blue:
    red = src_red.read(1).astype(float)
    green = src_green.read(1).astype(float)
    blue = src_blue.read(1).astype(float)
    sentinel_profile = src_red.profile

# Normalize for visualization
sentinel_rgb = np.stack([red, green, blue], axis=-1)
sentinel_rgb = sentinel_rgb / np.percentile(sentinel_rgb, 98) if np.percentile(sentinel_rgb, 98) > 0 else sentinel_rgb
sentinel_rgb = np.clip(sentinel_rgb, 0, 1)

# Visualize Sentinel-2 RGB composite
plt.figure(figsize=(8, 8))
plt.imshow(sentinel_rgb)
plt.title(f'Sentinel-2 RGB Composite - {sentinel_product_title}')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 5: Visualize Landsat RGB Composite

Load and visualize the RGB bands (B4, B3, B2 for Landsat 8/9) from the downloaded Landsat product.

In [ ]:
# Define paths to Landsat RGB bands (30m resolution)
landsat_product_dir = os.path.join(output_dir, landsat_product_id)
red_path = glob.glob(os.path.join(landsat_product_dir, '*_B4.TIF'))[0]
green_path = glob.glob(os.path.join(landsat_product_dir, '*_B3.TIF'))[0]
blue_path = glob.glob(os.path.join(landsat_product_dir, '*_B2.TIF'))[0]

# Load Landsat bands
with rasterio.open(red_path) as src_red, rasterio.open(green_path) as src_green, rasterio.open(blue_path) as src_blue:
    red = src_red.read(1).astype(float)
    green = src_green.read(1).astype(float)
    blue = src_blue.read(1).astype(float)
    landsat_profile = src_red.profile

# Normalize for visualization
landsat_rgb = np.stack([red, green, blue], axis=-1)
landsat_rgb = landsat_rgb / np.percentile(landsat_rgb, 98) if np.percentile(landsat_rgb, 98) > 0 else landsat_rgb
landsat_rgb = np.clip(landsat_rgb, 0, 1)

# Visualize Landsat RGB composite
plt.figure(figsize=(8, 8))
plt.imshow(landsat_rgb)
plt.title(f'Landsat RGB Composite - {landsat_product_id}')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 6: Save RGB Composites

Save the Sentinel-2 and Landsat RGB composites as GeoTIFF files.

In [ ]:
# Save Sentinel-2 RGB composite
sentinel_output_profile = sentinel_profile.copy()
sentinel_output_profile.update(count=3, dtype=rasterio.float32)
sentinel_rgb_path = os.path.join(output_dir, f'{sentinel_product_title}_rgb.tif')
with rasterio.open(sentinel_rgb_path, 'w', **sentinel_output_profile) as dst:
    dst.write(sentinel_rgb.transpose(2, 0, 1))

# Save Landsat RGB composite
landsat_output_profile = landsat_profile.copy()
landsat_output_profile.update(count=3, dtype=rasterio.float32)
landsat_rgb_path = os.path.join(output_dir, f'{landsat_product_id}_rgb.tif')
with rasterio.open(landsat_rgb_path, 'w', **landsat_output_profile) as dst:
    dst.write(landsat_rgb.transpose(2, 0, 1))

print(f'Sentinel-2 RGB composite saved to: {sentinel_rgb_path}')
print(f'Landsat RGB composite saved to: {landsat_rgb_path}')

## Next Steps

- Replace `aoi.geojson` with your own area of interest file.
- Update `your_scihub_username`, `your_scihub_password`, `your_earthexplorer_username`, and `your_earthexplorer_password` with your credentials.
- Adjust the date range or cloud cover thresholds to suit your needs.
- Proceed to the next notebook (`22_folium_visualization.ipynb`) for interactive visualization with Folium.

## Notes
- Ensure the AOI GeoJSON is valid and matches the desired study area.
- Sentinel-2 Level-2A products provide atmospherically corrected data; Landsat Collection 2 Level-2 provides surface reflectance.
- Check `docs/installation.md` for troubleshooting library installation.
- Landsat downloads may require authentication; ensure your EarthExplorer account is active.